In [1]:
SELECT @@SPID AS server_process_id, transaction_isolation_level
FROM [sys].[dm_exec_requests]
WHERE session_id = @@SPID

(1 row affected)

Total execution time: 00:00:00.047

client_id transaction_isolation_level 51 2

In [5]:
SELECT *
FROM [Bank].[AccountDetails]
WHERE acc_number = '7Z290387';

(1 row affected)

Total execution time: 00:00:00.039

id acc_number acc_holder_name balance 1 7Z290387 Ken 45

In [4]:
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;

Commands completed successfully.

Total execution time: 00:00:00.035

In [7]:
SELECT *
FROM [Bank].[AccountDetails]
WHERE acc_number = '7Z290387';

(1 row affected)

Total execution time: 00:00:00.037

id acc_number acc_holder_name balance 1 7Z290387 Ken 90

**Run the cell below after the transaction in ClientOne has been rolled back**

In [2]:
SELECT *
FROM [Bank].[AccountDetails]
WHERE acc_number = '7Z290387';

(1 row affected)

Total execution time: 00:00:00.331

id acc_number acc_holder_name balance 1 7Z290387 Ken 90

From this point, the focus shifts to Repeatable Reads

In [3]:
DECLARE @KenBalance int

BEGIN TRAN;

  SELECT @KenBalance = balance FROM [Bank].[AccountDetails] 
                               WHERE acc_holder_name = 'Ken';
  PRINT 'Initial balance for Ken: ' + CAST(@KenBalance AS VARCHAR(16));

  UPDATE [Bank].[AccountDetails]
  SET balance = balance - 45
  WHERE acc_holder_name = 'Ken';

  SELECT @KenBalance = balance FROM [Bank].[AccountDetails] 
                               WHERE acc_holder_name = 'Ken';
  PRINT 'Updated balance for Ken: ' + CAST(@KenBalance AS VARCHAR(16));

COMMIT TRAN;

Initial balance for Ken: 90

(1 row affected)

Updated balance for Ken: 45

Total execution time: 00:00:00.268

In [4]:
DECLARE @KenBalance int;

BEGIN TRAN;

  SELECT @KenBalance = balance FROM [Bank].[AccountDetails] 
                               WHERE acc_holder_name = 'Ken';
  PRINT 'Initial balance for Ken: ' + CAST(@KenBalance AS VARCHAR(16));

  UPDATE [Bank].[AccountDetails]
  SET balance = balance - 20
  WHERE acc_holder_name = 'Ken';

  SELECT @KenBalance = balance FROM [Bank].[AccountDetails] 
                               WHERE acc_holder_name = 'Ken';
  PRINT 'Updated balance for Ken: ' + CAST(@KenBalance AS VARCHAR(16));

COMMIT TRAN;

Initial balance for Ken: 45

: Msg 1222, Level 16, State 51, Line 9
Lock request time out period exceeded.

The statement has been terminated.

Updated balance for Ken: 45

Total execution time: 00:00:05.084

### Modifying another row poses no problem even while a lock is held on Ken's row.

In [5]:
DECLARE @TonyaBalance int

BEGIN TRAN;

  SELECT @TonyaBalance = balance FROM [Bank].[AccountDetails] 
                               WHERE acc_holder_name = 'Tonya';
  PRINT 'Initial balance for Tonya: ' + CAST(@TonyaBalance AS VARCHAR(16));

  UPDATE [Bank].[AccountDetails]
  SET balance = balance - 100
  WHERE acc_holder_name = 'Tonya';

  SELECT @TonyaBalance = balance FROM [Bank].[AccountDetails] 
                               WHERE acc_holder_name = 'Tonya';
  PRINT 'Updated balance for Tonya: ' + CAST(@TonyaBalance AS VARCHAR(16));

COMMIT TRAN;

Initial balance for Tonya: 200

(1 row affected)

Updated balance for Tonya: 100

Total execution time: 00:00:00.624